In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from get_users import scrape_books_from_user, main, scrape_users_from_book
from goodreads_scraper.get_books import scrape_book, get_all_editions
from selenium import webdriver
import json
import csv
import os
import glob

In [17]:
user_ids = set()
files = glob.glob('data/book_read_by_users_lists/*.json')
for file in files:
    with open(file, 'r') as f:
        users = json.load(f)
        user_ids.update(users.keys())
with open('./user_ids_to_scrape.txt', 'w') as f:
    for user_id in user_ids:
        f.write(user_id + '\n')

In [12]:
main(['--output_directory_path', 'data/user-read_lists', '--user_ids_path', './user_ids_to_scrape.txt'])

2023-01-08 19:05:19.601555 get_users.py: Scraping 3018343...
2023-01-08 19:05:19.601555 get_users.py: #710 out of 505696 users


In [3]:
from urllib.request import urlopen
from urllib.error import HTTPError
import bs4

In [3]:
user = json.load(open('data/user-read_lists/63409191_user-read_list.json'))

In [4]:
book_ids = [books['book_id'] for books in user]

In [18]:
book_ids = ['3300225', '333544']

In [3]:
# Read the cookies from the file
with open("goodreads_cookies.csv", "r") as f:
    cookies_string = f.read()

# Split the cookies string into a list of individual cookies
cookies = cookies_string.split("\n")

# Create a list to hold the selenium Cookies
selenium_cookies = []

# Iterate over the cookies, creating a selenium Cookie object for each one
for cookie in cookies[1:]: 
    name, value, domain, path = cookie.split(',')
    selenium_cookies.append({'name': name, 'value': value, 'domain': domain, 'path': path})

In [4]:
options = webdriver.FirefoxOptions()
options.add_argument('--headless')
driver = webdriver.Firefox()
driver.install_addon('C:\\Users\\thdub\\AppData\\Roaming\\Mozilla\\Firefox\\Profiles\\7r2u1d87.default-release\\extensions\\uBlock0@raymondhill.net.xpi')
driver.get("https://www.goodreads.com")
for cookie in selenium_cookies:
    driver.add_cookie(cookie)

In [7]:
# Add the cookies to the selenium driver
options = webdriver.ChromeOptions()
#options.add_argument('--headless')
options.add_argument('load-extension=C:\\Users\\thdub\\AppData\\Local\\Google\\Chrome\\User Data\\Default\\Extensions\\cjpalhdlnbpafiamejdnhcphjbkeiagm\\1.45.2_0')
driver = webdriver.Chrome(options=options)
driver.get("https://www.goodreads.com")
for cookie in selenium_cookies:
    driver.add_cookie(cookie)

In [5]:
#ffprofile = webdriver.FirefoxProfile('C:\\Users\\thdub\\AppData\\Roaming\\Mozilla\\Firefox\\Profiles\\7r2u1d87.default-release')
#driver = webdriver.Firefox(firefox_profile=ffprofile, options=options)

output_directory_path = 'data/book_read_by_users_lists'
books_file_path = 'data/user-read_lists/63409191_user-read_list.json'
args = {'output_directory_path': output_directory_path, 'books_file_path': books_file_path}

books = json.load(open(args['books_file_path']))
book_ids = [book['book_id'] for book in books]
books_already_scraped =  [file_name.replace('_book_read_by_users_list.json', '') for file_name in os.listdir(args['output_directory_path']) if file_name.endswith('.json') and not file_name.startswith('all_books')]
books_to_scrape       = [book_id for book_id in book_ids if book_id not in books_already_scraped]

print('already scraped {} out of {} books'.format(len(books_already_scraped), len(book_ids)))
num_scraped = len(books_already_scraped)
num_failed = 0

books_failed = []
for book_id in books_to_scrape:
    try:
        user_ids = scrape_users_from_book(book_id, driver)
        if user_ids:
            with open(os.path.join(output_directory_path, '{}_book_read_by_users_list.json'.format(book_id)), 'w') as f:
                json.dump(user_ids, f)
            num_scraped += 1
            print('scraped {} out of {} books'.format(num_scraped, len(book_ids)))
        else:
            print('book {} failed'.format(book_id))
            books_failed.append(book_id)
            num_failed += 1
    
    except Exception as e:
        print('exception {}'.format(e))
        print('book {} failed'.format(book_id))
        books_failed.append(book_id)
        num_failed += 1
    
driver.quit()

already scraped 185 out of 195 books
scraped 186 out of 195 books
scraped 187 out of 195 books
scraped 188 out of 195 books
scraped 189 out of 195 books
exception Message: Unable to locate element: .Overlay__window
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:182:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:394:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:275:16

book 32075671 failed
exception Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:182:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:394:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:275:16

book 15839976 failed
exception Message: Unable to locate element: .Overlay__window
Stacktrace:
RemoteError@chrome://re

KeyboardInterrupt: 

In [42]:
driver.quit()

In [3]:
import bs4
from urllib.request import urlopen
import json
import re
from time import sleep


In [5]:
url = 'https://www.goodreads.com/user/sign_in'
source = urlopen(url)
soup = bs4.BeautifulSoup(source, 'html.parser')
sign_in_fields = soup.find('div', attrs={'class': 'third_party_sign_in'})
sign_in_fields = sign_in_fields.find_all('a')
for sign_in_field in sign_in_fields:
    button = sign_in_field.find('button')
    if 'email' in button.text:
        login_url = sign_in_field['href']
        break

In [6]:
import urllib.request
import urllib
import http.cookiejar

# Set up a cookie jar to store the cookies sent by the website
cookie_jar = http.cookiejar.CookieJar()
opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cookie_jar))

# Log in to the website
login_data = urllib.parse.urlencode({'email': 'th.dubach@gmail.com', 'password': 'dXnZ472X9kydDIWcvDQV'}).encode('utf-8')
opener.open(login_url, login_data)

# Connect to a webpage while logged in
response = opener.open('https://www.goodreads.com/comment/list/63409191-thomas?ref=nav_profile_comment')
html = response.read()

In [5]:



# If you want to open Firefox
ffprofile = webdriver.FirefoxProfile('C:\\Users\\thdub\\AppData\\Roaming\\Mozilla\\Firefox\\Profiles\\7r2u1d87.default-release')
driver = webdriver.Firefox(firefox_profile=ffprofile)

book_url = 'https://www.goodreads.com/book/show/3300225/reviews'
driver.get(book_url)
sleep(5)

C:\Users\thdub\AppData\Local\Temp\ipykernel_12144\4207325300.py:9: DeprecationWarning: firefox_profile has been deprecated, please use an Options object
  ffprofile = webdriver.FirefoxProfile('C:\\Users\\thdub\\AppData\\Roaming\\Mozilla\\Firefox\\Profiles\\7r2u1d87.default-release')
C:\Users\thdub\AppData\Local\Temp\ipykernel_12144\4207325300.py:10: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  driver = webdriver.Firefox(firefox_profile=ffprofile)


In [59]:
elem = driver.find_element(By.ID, '__NEXT_DATA__')
match = re.search('"work":{"__ref":"Work:[^}]*', elem.get_attribute('innerHTML')).group()
work_url = match[match.find('kca'):-1]

In [ ]:
driver.req